In [ ]:
USER_NAME="yyyyy"
PASSWORD="xxxxx"
MAXVISITS=1000
MAXTIME=10
PONDERING="true"
# optional args
# supports: OPENCL, CUDA or AUTO
KATAGO_BACKEND="AUTO"
# supports: 40b, 30b, 20b, 40b-large or AUTO
WEIGHT_FILE="AUTO" 


import subprocess
if KATAGO_BACKEND == "AUTO":
  gpu_name=str(subprocess.check_output("nvidia-smi -q | grep \"Product Name\" | cut -d\":\" -f2 | tr -cd '[:alnum:]._-'", shell=True), encoding='utf-8')
  if gpu_name == "TeslaT4":
    KATAGO_BACKEND="CUDA"
  else:
    KATAGO_BACKEND="OPENCL"
!echo "Using Katago Backend: " $KATAGO_BACKEND

if WEIGHT_FILE == "AUTO":
  gpu_name=str(subprocess.check_output("nvidia-smi -q | grep \"Product Name\" | cut -d\":\" -f2 | tr -cd '[:alnum:]._-'", shell=True), encoding='utf-8')
  !echo "gpu_name: " $gpu_name
  if gpu_name == "TeslaK80" or gpu_name == "TeslaP4":
    WEIGHT_FILE="20b"
  else:
    WEIGHT_FILE="40b"

!echo "Using Katago Weight: " $WEIGHT_FILE

weight_urls = {
    '40b': 'https://github.com/lightvector/KataGo/releases/download/v1.4.5/g170-b40c256x2-s5095420928-d1229425124.bin.gz',
    '30b': 'https://github.com/lightvector/KataGo/releases/download/v1.4.5/g170-b30c320x2-s4824661760-d1229536699.bin.gz',
    '20b': 'https://github.com/lightvector/KataGo/releases/download/v1.4.5/g170e-b20c256x2-s5303129600-d1228401921.bin.gz',
    '40b-large': 'https://github.com/kinfkong/katago-colab/releases/download/v1.4.5/40b384.bin.gz'
}

# Install useful stuff
! apt install --yes ssh screen nano htop ranger git libzip4 1>/dev/null
! pip install oss2 1>/dev/null
%cd /content

!rm -rf katago-colab
!git clone https://github.com/kinfkong/katago-colab.git 1>/dev/null

#download the binarires
!wget --quiet https://github.com/kinfkong/katago-colab/releases/download/v1.4.5/katago-$KATAGO_BACKEND -O katago
!chmod +x /content/katago

!mkdir -p /root/.katago/
!cp -r /content/katago-colab/opencltuning /root/.katago/

#download the weights
weight_url = weight_urls[WEIGHT_FILE]
!wget --quiet $weight_url -O $WEIGHT_FILE".bin.gz" 
!rm -rf weight.bin.gz
!ln -s $WEIGHT_FILE".bin.gz" weight.bin.gz

%cd /content
!git clone https://github.com/kinfkong/zhang-katago.git 1>/dev/null
%cd /content/zhang-katago
!sed -i 's/^.*userName:.*$/ userName: $USER_NAME/g' ./conf.yaml
!sed -i 's/^.*password:.*$/ password: $PASSWORD/g' ./conf.yaml
!sed -i 's/^.*maxVisits = .*$/maxVisits = $MAXVISITS/g' /content/katago-colab/config/gtp_colab.cfg
!sed -i 's/^.*maxTime = .*$/maxTime = $MAXTIME/g' /content/katago-colab/config/gtp_colab.cfg
!sed -i 's/^.*ponderingEnabled = .*$/ponderingEnabled = $PONDERING/g' /content/katago-colab/config/gtp_colab.cfg
!cat ./conf.yaml
!cat /content/katago-colab/config/gtp_colab.cfg |grep maxTime
!cat /content/katago-colab/config/gtp_colab.cfg |grep maxVisits
!cat /content/katago-colab/config/gtp_colab.cfg |grep ponderingEnabled
!./start.sh & disown

!echo "done!"

Using Katago Backend:  OPENCL
gpu_name:  TeslaK80
Using Katago Weight:  20b




#以下的都是调试用的代码，普通用户可以忽略下面的所有内容

显示GPU类型

In [ ]:
!watch -n 1 nvidia-smi

In [ ]:
!/content/katago gtp -config /content/katago-colab/config/gtp_colab.cfg -model /content/weight.bin.gz

KataGo v1.4.5
Using TrompTaylor rules initially, unless GTP/GUI overrides this
Loaded config /content/katago-colab/config/gtp_colab.cfg
Loaded model /content/weight.bin.gz
Model name: g170-b40c256x2-s5095420928-d1229425124
GTP ready, beginning main protocol loop
^C
